In [4]:
#Import libs 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.metrics.pairwise import euclidean_distances
import random
import mlflow
import mlflow.sklearn
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 18)

In [135]:
core_data_init = pd.read_csv('ACG_CORE_por_k.csv')
core_data = core_data_init[1:].copy()
core_data_bal = core_data[core_data.CFORMATION.str.contains('Balakhany')]
core_data_bal['wellName'] = core_data_bal['wellName'].astype('string')
core_data_bal['DEPTH'] = core_data_bal['DEPTH'].astype('float')
core_data_bal['DEPTH'] = core_data_bal.DEPTH.round(2)
core_data_bal['H_CKHEL'] = core_data_bal['H_CKHEL'].astype('float')
core_data_bal['H_CKHEL_AMB'] = core_data_bal['H_CKHEL_AMB'].astype('float')
core_data_bal['H_CPOR'] = core_data_bal['H_CPOR'].astype('float')
core_data_bal['H_CPOR_AMB'] = core_data_bal['H_CPOR_AMB'].astype('float')
core_data_bal['CFORMATION'] = core_data_bal['CFORMATION'].astype('string')
core_data_bal = core_data_bal.fillna(0)
core_data_bal = core_data_bal.replace(-9999, 0)
core_data_bal = core_data_bal.replace('-9999', '0')
core_data_bal_sel = core_data_bal[core_data_bal.H_CPOR > 0]
# core_data_bal_sel.head()

In [6]:
df_prq = pd.read_parquet('ACG_wells_JOINT_BEST_v6.parquet.gzip')

In [102]:
log_core = df_prq.set_index(['wellName','DEPTH']).join(core_data_bal_sel.set_index(['wellName','DEPTH'])).reset_index()

In [ ]:
log_core_sel = log_core[['wellName', 'DEPTH', 'FORMATION', 'CFORMATION', 
                        'LPERM', 'PHIT', 'H_CKHEL', 'H_CKHEL_AMB', 'H_CPOR', 'H_CPOR_AMB',
                        'V_CKHEL', 'V_CKHEL_AMB', 'V_CPOR', 'V_CPOR_AMB']]
log_core_sel = log_core_sel[(log_core_sel.H_CPOR>13) & (log_core_sel.PHIT>0.13) &
                            (log_core_sel.H_CKHEL > 10) & (log_core_sel.LPERM > 10)]
# log_core_sel        
sca = px.scatter(log_core_sel, x='H_CPOR', y='PHIT',color='wellName')
ln = px.line(x=[10,30], y=[0.1,0.3])
ln.update_traces(line=dict(color = 'red'))
fig = go.Figure(data = ln.data + sca.data)
fig.update_layout(title = 'Comparison PHIT-log vs CPOR',width=600,height=600, xaxis_title="H_CPOR", yaxis_title="PHIT",
                    margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(xaxis_range=[10,35], yaxis_range=[0.1,0.35])
fig.show()

In [138]:
def well_phit_bal8_10(wellname):
    data = log_core[(log_core.wellName==wellname)]
    y8 = data[data.FORMATION=='Balakhany VIII sand']['DEPTH']
    x1_8 = data[data.FORMATION=='Balakhany VIII sand']['PHIT']
    x2_8 = data[data.FORMATION=='Balakhany VIII sand']['H_CPOR']
    y10 = data[data.FORMATION=='Balakhany X sand']['DEPTH']
    x1_10 = data[data.FORMATION=='Balakhany X sand']['PHIT']
    x2_10 = data[data.FORMATION=='Balakhany X sand']['H_CPOR']  
    fig, ax = plt.subplots(1,2, figsize=(6,7), sharey=False)
    ax[0].plot(x1_8, y8, color='green', label='PHIT'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 0.3)
    ax[1].plot(x1_10, y10, color='green', label='PHIT'), ax[1].invert_yaxis(), ax[1].set_xlim(0, 0.3)
    twin1 = ax[0].twiny() 
    twin1.scatter(x2_8, y8, color='blue', label='H_CKHEL', alpha=.5)
    twin1.set_xlim(0, 30)
    twin2 = ax[1].twiny() 
    twin2.scatter(x2_10, y10, color='blue', label='H_CKHEL', alpha=.5)
    twin2.set_xlim(0, 30)
    ax[0].title.set_text('Bal VIII sand')
    ax[1].title.set_text('Bal X sand')
    ax[0].grid()
    ax[1].grid()
    fig.suptitle(wellname, fontsize=14)
    fig.legend(fontsize=8)
    fig.tight_layout()

In [ ]:
for well in core_data_bal_sel.wellName.unique():
    well_phit_bal8_10(well)

In [ ]:
ln = px.line(x=[1,2000], y=[1,2000])
sca = px.scatter(log_core_sel, x='H_CKHEL', y='LPERM', color='wellName')
ln.update_traces(line=dict(color = 'red', width=4))
fig = go.Figure(data = ln.data + sca.data)
fig.update_layout(title = 'Comparison PERM-log vs CKHEL',width=600,height=600, xaxis_title="H_CKHEL", yaxis_title="LPERM",
                    margin=dict(l=10,r=10,b=10,t=40))
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.show()

In [171]:
def well_perm_bal8_10(wellname):
    data = log_core[(log_core.wellName==wellname)]
    y8 = data[data.FORMATION=='Balakhany VIII sand']['DEPTH']
    x1_8 = data[data.FORMATION=='Balakhany VIII sand']['LPERM']
    x2_8 = data[data.FORMATION=='Balakhany VIII sand']['H_CKHEL']
    y10 = data[data.FORMATION=='Balakhany X sand']['DEPTH']
    x1_10 = data[data.FORMATION=='Balakhany X sand']['LPERM']
    x2_10 = data[data.FORMATION=='Balakhany X sand']['H_CKHEL']  
    fig, ax = plt.subplots(1,2, figsize=(6,7), sharey=False)
    ax[0].plot(x1_8, y8, color='#8c12b8', label='PHIT'), ax[0].invert_yaxis(), ax[0].set_xlim(0.1, 1000)
    ax[1].plot(x1_10, y10, color='#8c12b8', label='PHIT'), ax[1].invert_yaxis(), ax[1].set_xlim(0.1, 1000)
    ax[0].set_xscale('log'), ax[1].set_xscale('log')
    twin1 = ax[0].twiny() 
    twin1.scatter(x2_8, y8, color='red', label='H_CKHEL', alpha=.5)
    twin1.set_xlim(0.1, 1000), twin1.set_xscale('log')
    twin2 = ax[1].twiny() 
    twin2.scatter(x2_10, y10, color='red', label='H_CKHEL', alpha=.5)
    twin2.set_xlim(0.1, 1000), twin2.set_xscale('log')
    ax[0].title.set_text('Bal VIII sand')
    ax[1].title.set_text('Bal X sand')
    ax[0].grid(which = 'both', color='#6e6e6e', linewidth=0.1)
    ax[1].grid(which = 'both', color='#6e6e6e', linewidth=0.1)
    fig.suptitle(wellname, fontsize=14)
    fig.legend(fontsize=8)
    fig.tight_layout()

In [ ]:
for well in core_data_bal_sel.wellName.unique()[:]:
    well_perm_bal8_10(well)